In [2]:
import pandas as pd
from IPython.display import display

ratings_movies = pd.read_csv('data_12/ratings_movies.csv')
# наш результат объединения таблиц с рейтингами
display(ratings_movies.head())

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Для решения задач нам понадобится выделить из признака title год выпуска фильма. Для этого напишем функцию get_year_release(arg).

In [7]:
import re # библиотека для регулярных выражений
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

1. Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
У скольких фильмов не указан год их выпуска?

In [15]:
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
display(ratings_movies.head())

,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995.0
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995.0
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995.0
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995.0


In [16]:
print(ratings_movies['year_release'].shape[0] - ratings_movies['year_release'].count())

18


2. Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
В качестве ответа запишите название этого фильма без указания года его выпуска.

In [35]:
df_1999 = ratings_movies['year_release']==1999
ratings_movies[df_1999].groupby('title')['rating'].mean().sort_values()

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

3. Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов)

In [36]:
mask = ratings_movies['year_release']==2010
ratings_movies[mask].groupby('genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

4. Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) жанров (genres) фильмов? В качестве ответа запишите идентификатор этого пользователя.

In [45]:
ratings_movies.groupby('userId')['genres'].nunique().sort_values(ascending=False)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

5. Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
В качестве ответа укажите идентификатор этого пользователя.
Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), можно воспользоваться методом agg() на сгруппированных данных.

In [55]:
ratings_movies.groupby('userId')['rating'].agg(
    ['count', 'mean']
    ).sort_values(['count', 'mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


6. Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг (среднее по столбцу rating), и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [73]:
mask = ratings_movies['year_release']==2018
rating_2018 = ratings_movies[mask].groupby('genres')['rating'].agg(
    ['mean', 'count']
).sort_values(['mean'], ascending=False)
print(rating_2018[rating_2018['count']>10])


                             mean  count
genres                                  
Action|Adventure|Sci-Fi  3.928571     14
Action|Comedy|Sci-Fi     3.875000     12


7. Добавьте в таблицу новый признак year_rating  — год выставления оценки. Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра. Выберите верные варианты ответа, исходя из построенной таблицы:
A За весь период (с 1996 по 2018 год) сочетание жанров Action|Adventure ни разу не получало среднюю оценку ниже 3.
B Наилучшую оценку жанр Action|Adventure|Animation|Children|Comedy|IMAX получил в 2010 году.
C Среди сочетаний жанров, получивших наивысшую среднюю оценку в 2018 году, есть сочетание Animation|Children|Mystery.
D Для жанра Comedy прослеживается тенденция падения рейтинга с каждым годом (с 1996 по 2018).

In [4]:
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'])

In [9]:
ratings_year = ratings_movies['date'].dt.year
ratings_movies['year_rating'] =  ratings_year
display(ratings_movies)

,Unnamed: 0,userId,movieId,rating,date,title,genres,year_rating
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,2000
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,2000
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,2000
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,2000
...,...,...,...,...,...,...,...,...
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller,2017
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller,2017
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror,2017
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi,2017


In [33]:
new_df = ratings_movies.pivot_table(
    values='rating',
    index='year_rating',
    columns='genres',
    aggfunc='mean'
)
display(new_df)

genres,(no genres listed),Action,Action|Adventure,Action|Adventure|Animation,Action|Adventure|Animation|Children,Action|Adventure|Animation|Children|Comedy,Action|Adventure|Animation|Children|Comedy|Fantasy,Action|Adventure|Animation|Children|Comedy|IMAX,Action|Adventure|Animation|Children|Comedy|Romance,Action|Adventure|Animation|Children|Comedy|Sci-Fi,...,Romance|Thriller,Romance|War,Romance|Western,Sci-Fi,Sci-Fi|IMAX,Sci-Fi|Thriller,Sci-Fi|Thriller|IMAX,Thriller,War,Western
year_rating,,,,,,,,,,,,,,,,,,,,,
1996,NaN,2.730769,3.454545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.666667,NaN,3.838095,NaN,3.117647
1997,NaN,3.538462,4.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.400000,NaN,3.923077,NaN,3.000000
1998,NaN,NaN,4.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.800000,NaN,NaN
1999,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000,NaN,NaN,NaN,NaN,4.000000,NaN,3.700000,4.5,4.000000
2000,NaN,2.588235,3.738462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.000,NaN,3.0,3.416667,NaN,2.142857,NaN,3.087912,3.0,4.058824
2001,NaN,3.000000,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,2.500000,NaN,2.500000,NaN,3.477273,3.0,3.111111
2002,NaN,2.750000,4.304348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000,NaN,NaN,3.750000,NaN,3.600000,NaN,3.583333,3.5,3.000000
2003,NaN,3.833333,3.277778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.375,2.5,NaN,2.333333,NaN,3.142857,NaN,3.250000,3.0,4.000000
2004,NaN,2.700000,4.136364,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,...,3.000,3.0,3.5,2.125000,NaN,NaN,NaN,3.464286,3.0,3.800000


In [39]:
new_df[new_df['Action|Adventure']<3].shape[0] # проверка условия А, ответ: да

0

In [42]:
new_df['Action|Adventure|Animation|Children|Comedy|IMAX'].sort_values(ascending=False).head(3)
# условие В, ответ: нет

year_rating
2013    5.000000
2018    3.700000
2016    3.666667
Name: Action|Adventure|Animation|Children|Comedy|IMAX, dtype: float64

In [44]:
new_df['Animation|Children|Mystery'].sort_values(ascending=False).head(3)
# условие С, ответ: да

year_rating
2018    5.0
1996    NaN
1997    NaN
Name: Animation|Children|Mystery, dtype: float64

In [49]:
new_df['Comedy'] # условие D, ответ: нет

year_rating
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: Comedy, dtype: float64

## Заказы в интернет-магазине

In [52]:
import pandas as pd
from IPython.display import display

orders_df = pd.read_csv('data_12/orders.csv', sep=';')
display(orders_df)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3


In [54]:
products_df = pd.read_csv('data_12/products.csv', sep=';')
display(products_df)

,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR
5,86,"Носки Простые, муж",45,RUR
6,91,"Носки Честные, муж",50,RUR
7,103,"Носки Подарочные, муж",199,RUR
8,104,"Носки Подарочные, жен",249,RUR
9,124,Носки беговые Camino,999,RUR


Объедините заданные таблицы в таблицу orders_products, чтобы в результирующей таблице оказалась информация обо всех заказах, но не оказалось информации о продуктах, на которых заказов ещё не поступало. 

In [57]:
orders_products = orders_df.merge(
    products_df,
    how='left',
    left_on='ID товара',
    right_on='Product_ID'
)
display(orders_products)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86.0,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104.0,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104.0,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104.0,"Носки Подарочные, жен",249.0,RUR
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124.0,Носки беговые Camino,999.0,RUR
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,91.0,"Носки Честные, муж",50.0,RUR
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,103.0,"Носки Подарочные, муж",199.0,RUR
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,104.0,"Носки Подарочные, жен",249.0,RUR


Какой идентификатор (Order ID) имеет заказ, для которого не оказалось информации о товаре?

In [59]:
orders_products.tail(1)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
17,01.01.2001 00:00:00,0,1,"Оплачен, формируется к отправке",Да,Нет,Нет,666,1,NaN,NaN,NaN,NaN


На какой товар была произведена отмена?
В качестве ответа запишите название этого товара (Name).

In [60]:
orders_products[orders_products['Отменен']=='Да']

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124.0,Носки беговые Camino,999.0,RUR


Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
В ответ запишите идентификатор этого покупателя (ID Покупателя).
Прибыль состоит только из оплаченных заказов и рассчитывается как количество купленного товара, умноженное на его цену.

In [65]:
paid = orders_products[orders_products['Оплачен']=='Да']
paid_max = (paid['Количество']*paid['Price']).max()
orders_products[orders_products['Количество']*orders_products['Price']==paid_max]

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
10,08.11.2019 08:36:21,2,7,Выполнен,Да,Нет,Да,74,1,74.0,Платье Ночная Жизнь XXXL,8999.0,RUR
